# Demo of Neuroimaging Python Reading in of Files

In [1]:
import numpy as np
import os
import json
import nibabel as nb
import pandas as pd

import scipy.io

from nibabel.affines import apply_affine

# import basic plotting
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

from pprint import pprint

In [2]:
lut_filepath = os.path.join(output_results, 'multilevel_lookup_table.txt')

lut_df = pd.read_csv(lut_filepath, delimiter="\t", index_col=0)
colnames = lut_df.columns

# map nan elements to some empty str
lut_df.isna

# for col in colnames:
#     print(col, np.unique(lut_df[col]))
# display(lut_df)


NameError: name 'output_results' is not defined

In [ ]:
# get a list of the type of atlases
atlas_list = lut_df.columns


# MRI Cloud Results

In [3]:
patid = "umf001"

datadir = "/Users/adam2392/Downloads/neuroimage_data/la02_mricloud_result/"
datadir = f"/home/adam2392/Documents/Dropbox/phd_research/data/neuroimaging_results/mricloud_results/{patid}"
output_results = os.path.join(datadir, 'output')

check_files = os.listdir(datadir)
datafiles = os.listdir(output_results)

print("Datafiles from mricloud: ")
pprint(check_files)
pprint(datafiles)


Datafiles from mricloud: 
['priority.txt',
 'identity.txt',
 'output',
 'config.txt',
 'local_address.txt',
 'atlas_list.txt']
['matrix.txt',
 'mni.imgsize',
 'umf001_t1.img',
 'umf001_t1_MNI.hdr',
 'multilevel_lookup_table.txt',
 'umf001_t1_286Labels_MNI.img',
 'umf001_t1_286Labels.nii',
 'umf001_t1.imgsize',
 'umf001_t1_286Labels_corrected_stats.txt',
 'umf001_t1.hdr',
 'umf001_t1_7Labels_MNI.img',
 'matrix_air.txt',
 'umf001_t1_7Labels.hdr',
 'umf001_t1_7Labels_MNI.hdr',
 'umf001_t1_7Labels.img',
 'umf001_t1_286Labels.hdr',
 'umf001_t1_286Labels.img',
 'umf001_t1_286Labels_MNI_stats.txt',
 'umf001_t1_MNI.img',
 'umf001_t1_286Labels_MNI.hdr']


In [4]:
# read the img/hdr file
# read in header
hdr = nb.load(os.path.join(output_results, f'{patid}_t1_286Labels.hdr'))

# img
img = nb.load(os.path.join(output_results, f'{patid}_t1_286Labels.img'))

img.hdr = hdr

# convert to nifti
nb.save(img, os.path.join(output_results, f'{patid}_t1_286Labels.nii'))

# print(hdr)
imgdata = img.get_fdata()

print(np.unique(imgdata))
# print(imgdata)

[  0.   1.   2.   3.   4.   5.   6.   7.   8.   9.  10.  11.  12.  13.
  14.  15.  16.  17.  18.  19.  20.  21.  22.  23.  24.  25.  26.  27.
  28.  29.  30.  31.  32.  33.  34.  35.  36.  37.  38.  39.  40.  41.
  42.  43.  44.  45.  46.  47.  48.  49.  50.  51.  52.  53.  54.  55.
  56.  57.  58.  59.  60.  61.  62.  63.  64.  65.  66.  67.  68.  69.
  70.  71.  72.  73.  74.  75.  76.  77.  78.  79.  80.  81.  82.  83.
  84.  85.  86.  87.  88.  89.  90.  91.  92.  93.  94.  95.  96.  97.
  98.  99. 100. 101. 102. 103. 104. 105. 106. 107. 108. 109. 110. 111.
 112. 113. 114. 115. 116. 117. 118. 119. 120. 121. 122. 123. 124. 125.
 126. 127. 128. 129. 130. 131. 132. 133. 134. 135. 136. 137. 138. 139.
 140. 141. 142. 143. 144. 145. 146. 147. 148. 149. 150. 151. 152. 153.
 154. 155. 156. 157. 158. 159. 160. 162. 163. 164. 165. 166. 167. 168.
 169. 170. 171. 172. 173. 174. 175. 176. 177. 178. 179. 180. 181. 182.
 183. 184. 185. 186. 187. 188. 189. 190. 191. 192. 193. 194. 195. 196.
 197. 

In [5]:
aff_mat = img.affine

# inverse affine
anat2vox = np.linalg.inv(aff_mat)#.dot(aff_mat)

print(anat2vox.shape)

(4, 4)


In [10]:
print(img.header['pixdim'])

[0.      1.      0.46875 0.46875 0.      0.      0.      0.     ]


In [52]:

class MatReader():
    '''
    Object to read mat files into a nested dictionary if need be.
    Helps keep strucutre from matlab similar to what is used in python.
    '''

    def __init__(self, filename=None):
        self.filename = filename

    def loadmat(self, filename):
        '''
        this function should be called instead of direct spio.loadmat
        as it cures the problem of not properly recovering python dictionaries
        from mat files. It calls the function check keys to cure all entries
        which are still mat-objects
        '''
        data = scipy.io.loadmat(
            filename,
            struct_as_record=False,
            squeeze_me=True)
        return self._check_keys(data)

    def _check_keys(self, dict):
        '''
        checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        '''
        for key in dict:
            if isinstance(dict[key], scipy.io.matlab.mio5_params.mat_struct):
                dict[key] = self._todict(dict[key])
        return dict

    def _todict(self, matobj):
        '''
        A recursive function which constructs from matobjects nested dictionaries
        '''
        dict = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, scipy.io.matlab.mio5_params.mat_struct):
                dict[strg] = self._todict(elem)
            elif isinstance(elem, np.ndarray):
                dict[strg] = self._tolist(elem)
            else:
                dict[strg] = elem
        return dict

    def _tolist(self, ndarray):
        '''
        A recursive function which constructs lists from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        '''
        elem_list = []
        for sub_elem in ndarray:
            if isinstance(sub_elem, scipy.io.matlab.mio5_params.mat_struct):
                elem_list.append(self._todict(sub_elem))
            elif isinstance(sub_elem, np.ndarray):
                elem_list.append(self._tolist(sub_elem))
            else:
                elem_list.append(sub_elem)
        return elem_list

    def convertMatToJSON(self, matData, fileName):
        jsonData = {}

        for key in matData.keys():
            if (type(matData[key])) is np.ndarray:
                serializedData = pickle.dumps(
                    matData[key], protocol=0)  # protocol 0 is printable ASCII
                jsonData[key] = serializedData
            else:
                jsonData[key] = matData[key]

        with contextlib.closing(bz2.BZ2File(fileName, 'wb')) as f:
            json.dump(jsonData, f)
elecsdict = MatReader().loadmat(elecfile)
elecmatrix = elecsdict['elecmatrix']
eleclabels = elecsdict['eleclabels']
elecanat = elecsdict['anatomy']

eleclabels = [x[0] for x in eleclabels]
elecanat = [x[3] for x in elecanat]

# print(elecanat)
# print(elecmatrix)
print(eleclabels)
print(elecsdict.keys())

['btm1', 'btm2', 'btm3', 'btm4', 'btm5', 'btm6', 'btp1', 'btp2', 'btp3', 'btp4', 'btp5', 'btp6', 'c1', 'c2', 'c3', 'c4', 'c5', 'c6', 'c7', 'c8', 'c9', 'c10', 'c11', 'c12', 'c13', 'c14', 'c15', 'c16', 'c17', 'c18', 'c19', 'c20', 'c21', 'c22', 'c23', 'c24', 'c25', 'c26', 'c27', 'c28', 'c29', 'c30', 'c31', 'c32', 'c33', 'c34', 'c35', 'c36', 'c37', 'c38', 'c39', 'c40', 'c41', 'c42', 'c43', 'c44', 'c45', 'c46', 'c47', 'c48', 'c49', 'c50', 'c51', 'c52', 'c53', 'c54', 'c55', 'c56', 'c57', 'c58', 'c59', 'c60', 'c61', 'c62', 'c63', 'c64']
dict_keys(['__header__', '__version__', '__globals__', 'anatomy', 'eleclabels', 'elecmatrix'])


In [54]:
# read in elecs file
elecfile = f"/home/adam2392/hdd/data/neuroimaging/freesurfer_output/{patid}/elecs/{patid}_elecs_all.mat"

elecsdict = scipy.io.loadmat(elecfile, )
elecmatrix = elecsdict['elecmatrix']
eleclabels = elecsdict['eleclabels']

# print(elecmatrix)
# print(eleclabels)
print(elecsdict.keys())

dict_keys(['__header__', '__version__', '__globals__', 'anatomy', 'eleclabels', 'elecmatrix'])


In [58]:
canon = nb.as_closest_canonical(img)
print(nb.aff2axcodes(canon.affine))
print(nb.aff2axcodes(img.affine))

('R', 'A', 'S')
('L', 'A', 'S')


In [56]:
elecvox = [np.round(apply_affine(anat2vox, x)) for x in elecmatrix]

print(elecvox)

[array([ 41., 186., 166.]), array([ 34., 196., 162.]), array([ 27., 207., 159.]), array([ 16., 219., 160.]), array([  7., 233., 162.]), array([  1., 245., 171.]), array([ 32., 131., 169.]), array([ 26., 149., 166.]), array([ 20., 164., 166.]), array([ 12., 183., 166.]), array([  6., 199., 166.]), array([ -0., 217., 172.]), array([ 52., 408., 289.]), array([ 34., 390., 302.]), array([ 25., 372., 310.]), array([ 23., 352., 320.]), array([ 21., 330., 328.]), array([ 19., 308., 338.]), array([ 18., 288., 346.]), array([ 19., 266., 354.]), array([ 35., 403., 270.]), array([ 22., 382., 284.]), array([ 18., 363., 294.]), array([ 15., 341., 303.]), array([ 13., 322., 311.]), array([ 11., 300., 320.]), array([ 10., 279., 329.]), array([ 10., 260., 337.]), array([ 24., 394., 254.]), array([ 18., 375., 261.]), array([ 14., 353., 271.]), array([ 11., 333., 281.]), array([  8., 312., 293.]), array([  6., 293., 300.]), array([  4., 271., 310.]), array([  3., 252., 318.]), array([ 20., 377., 238.]), 

# Convert Coordinate System

In [86]:
orig_t1 = f"/home/adam2392/hdd/data/neuroimaging/freesurfer_output/{patid}/mri/T1.nii.gz"
orig_img = nb.load(orig_t1)

# get original orientation
orig_code = nb.aff2axcodes(orig_img.affine)

# end orientation
end_code = nb.aff2axcodes(img.affine)

ornt_orig = nb.orientations.axcodes2ornt(orig_code)
ornt_end = nb.orientations.axcodes2ornt(end_code)

# transform start to end
ornt_start_to_end = nb.orientations.ornt_transform(ornt_orig, ornt_end)

# affine start to end
aff_start_end = nb.orientations.inv_ornt_aff(ornt_start_to_end, shape=img.shape)

new_img_data = nb.orientations.apply_orientation(imgdata, ornt_start_to_end)

# new affine
newaff = aff_start_end.dot(img.affine)
# newaff = img.affine.dot(aff_start_end)
newcode = nb.aff2axcodes(newaff)

print("Old code: ", orig_code)
print("New code: ", end_code)
print(nb.aff2axcodes(newaff))
# print(newcode)

print(imgdata.shape)
print(new_img_data.shape)

Old code:  ('L', 'I', 'P')
New code:  ('L', 'A', 'S')
('L', 'I', 'P')
(117, 512, 512, 1)
(117, 512, 512, 1)


# Convert Voxels into XYZ Space

In [ ]:
def f(i, j, k):
    """ Return X, Y, Z coordinates for i, j, k """
    return M.dot([i, j, k]) + abc
from nibabel.affines import apply_affine

# for i in range()
# apply_affine(img.affine, )


In [6]:
imgtypes = [
    '',
    '_7Labels',
    '_7Labels_MNI',
    '_286Labels',
    '_286Labels_MNI',
    '_MNI'
]

In [8]:
imgsizes = [
    'mni',
#     'la02_t1_image'
]

for _imgsize in imgsizes:
    # img-size
    with open(os.path.join(output_results, '{}.imgsize'.format(_imgsize)), 'r') as f:
        imgsize = f.read()
    print(imgsize)

********************************************
THIS FILE HAS IMAGE SIZE INFORMATION FOR ANALYZE IMG FILE
/work/00303/tg455757/stampede2/M1_101214/atlassets/MNI/JHU_MNI_T1.img
********************************************
1) Nx, Ny, Nz : IMGSIZE
181	217	181



In [34]:
# read in header
hdr = nb.load(os.path.join(output_results, 'la02_t1_image_7Labels.hdr'))

# img-size
with open(os.path.join(output_results, 'la02_t1_image.imgsize'), 'r') as f:
    imgsize = f.read()
# print(imgsize)

# img
img = nb.load(os.path.join(output_results, 'la02_t1_image_7Labels.img'))
orientation = nb.aff2axcodes(img.affine)

# read hdr, img in loop
for imgtype in imgtypes:
    # read in header
    hdr = nb.load(os.path.join(output_results, 'la02_t1_image{}.hdr'.format(imgtype)))

    # img
    img = nb.load(os.path.join(output_results, 'la02_t1_image{}.img'.format(imgtype)))
    orientation = nb.aff2axcodes(img.affine)

    print(imgtype, orientation)

 ('L', 'A', 'S')
_7Labels ('L', 'A', 'S')
_7Labels_MNI ('L', 'A', 'S')
_286Labels ('L', 'A', 'S')
_286Labels_MNI ('L', 'A', 'S')
_MNI ('L', 'A', 'S')


In [37]:
print(hdr)

<class 'nibabel.spm2analyze.Spm2AnalyzeImage'>
data shape (181, 217, 181, 1)
affine: 
[[  -1.    0.    0.   90.]
 [   0.    1.    0. -108.]
 [   0.    0.    1.  -90.]
 [   0.    0.    0.    1.]]
metadata:
<class 'nibabel.spm2analyze.Spm2AnalyzeHeader'> object, endian='<'
sizeof_hdr     : 348
data_type      : b'16 bits'
db_name        : b'DICOM->Analyze7.5'
extents        : 16384
session_error  : 0
regular        : b'r'
hkey_un0       : b''
dim            : [  4 181 217 181   1   0   0   0]
vox_units      : b'mm'
cal_units      : b'  '
unused1        : 0
datatype       : float32
bitpix         : 32
dim_un0        : 0
pixdim         : [0. 1. 1. 1. 0. 0. 0. 0.]
vox_offset     : 0.0
scl_slope      : nan
scl_inter      : 0.0
funused3       : 0.0
cal_max        : 0.0
cal_min        : 0.0
compressed     : 0
verified       : 0
glmax          : 255
glmin          : 0
descrip        : b'Dcm2Analyze 2.0, Author: Hangyi Jiang'
aux_file       : b'see: *.dat/*.txt files'
orient         : b''
origin 

In [38]:
# This contains structure look-up table in a text format.
lut_file = 'multilevel_lookup_table.txt',
matairfile = 'matrix_air.txt'
matfile = 'matrix.txt',
corrected_stats_file = 'la02_t1_image_286Labels_corrected_stats.txt',

# Freesurfer Output Results

In [39]:
patient = 'la03'

# define freesurfer outputdir
freesurfer_dir = ""

# define the subject directory
subj_dir = os.path.join("", patient)

In [40]:
# define volume file
desikan_file = os.path.join(subj_dir, 'aparc+aseg.mgz')
destrieux_file = os.path.join(subj_dir, 'aparc.a2009s+aseg.mgz')

In [ ]:
# read in volume files
desikan_vol = nb.load(desikan_file)
destrieux_vol = nb.load(destrieux_file)
